In [ ]:
import cv2
import numpy as np

def extract_signature(image_path, crop_margins=(0, 55, 0, 20)):
    """
    Extracts and refines a signature from a passport image, then applies final cropping.
    
    Args:
        image_path (str): Path to the passport image.
        crop_margins (tuple): (top, bottom, left, right) pixels to crop from the final signature.
                             Example: (5, 5, 10, 10) removes 5px from top/bottom, 10px from left/right.
    
    Returns:
        np.ndarray: The cleaned and cropped signature (black-on-white), or None if not found.
    """
   
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    denoised = cv2.bilateralFilter(gray, 9, 75, 75)
    thresh = cv2.adaptiveThreshold(
        denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 4
    )
    height, width = thresh.shape
    roi = thresh[int(height*0.7):height, int(width*0.6):width]
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None

    combined = np.vstack(contours)
    x, y, w, h = cv2.boundingRect(combined)
    x += int(width * 0.6)
    y += int(height * 0.7)
    signature_region = gray[y:y+h, x:x+w]
    _, binary = cv2.threshold(signature_region, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    vertical_projection = np.sum(binary, axis=1) / 255
    threshold = 0.1 * np.max(vertical_projection)
    last_row = np.where(vertical_projection > threshold)[0][-1]
    refined_signature = binary[:last_row+1, :]
    inverted = cv2.bitwise_not(refined_signature)  # Black-on-white

    top, bottom, left, right = crop_margins
    h, w = inverted.shape
    # Ensure crop values don't exceed image dimensions
    top = min(top, h-1)
    bottom = min(bottom, h-1)
    left = min(left, w-1)
    right = min(right, w-1)
    # Apply cropping
    cropped = inverted[top:h-bottom, left:w-right]

    return cropped

# Example usage
signature = extract_signature("passport.png")
if signature is not None:
    cv2.imwrite("signature_final.png", signature)
    print("Signature extracted and cropped successfully!")
else:
    print("No signature found.")